# Unificación de datos

El objetivo de este Notebook, será unificar los datos que vienen esparcidos en varios JSON en distintas carpetas proporcionadas, esto con el objetivo de poder trabajar en un solo DataFrame todos los datos.

# Importamos las librerías necesarias

In [1]:
import pandas as pd # Trabajaremos datos con Pandas.
import os # Trabajaremos con las carpetas del sistema operativo con OS.
import json # Leeremos los archivos en grupo con json.

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_7876\4128522967.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd # Trabajaremos datos con Pandas.


# Unificamos los datos en un solo DataFrame

## 1. Función para leer carpetas

Creamos una función para nuestro objetivo, el cual será leer cada archivo JSON dentro de las carpetas y retornar un DataFrame unificado de todos ellos.

In [2]:
def leer_carpeta(path):
    '''
    Leer una carpeta de Json.

    Lee los archivos JSON de una carpeta, y los unifica en un solo DataFrame de pandas.

    Requiere importar:
    pandas
    json
    os

    Recibe:
    La ruta hacia una carpeta que contenga archivos JSON.

    Retorna:
    Un DataFrame de todos los JSON.
    '''
    total_data = [] # Guardamos los datos que vayamos leyendo.
    # Iteramos en la carpeta.
    for file in os.listdir(path):
        filepath = os.path.join(path, file) # Creamos la ruta hacia el archivo sobre el que estamos iterando.
        with open(filepath, 'r') as f: # Abrimos el archivo.
            for line in f: # Iteramos sobre sus lineas
                data = json.loads(line) # Extraemos la linea como un JSON.
                total_data.append(data) # Guardamos el JSON en una lista.
    return pd.DataFrame(total_data) # Creamos un DataFrame con los datos finales y lo retornamos.

## 1.2 Leemos cada DataFrame

In [3]:
mainpath = '../../Data/Google/review-' # Ruta base a la cual le agregaremos el nombre del estado que usaremos.

In [4]:
df_cal = leer_carpeta(mainpath + 'California') # Leemos la carpeta agregando el nombre del estado y lo almacenamos en un DataFrame.

Previsualizamos la estructura del DataFrame para tener una guía.

In [5]:
df_cal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2700000 entries, 0 to 2699999
Data columns (total 8 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   user_id  object
 1   name     object
 2   time     int64 
 3   rating   int64 
 4   text     object
 5   pics     object
 6   resp     object
 7   gmap_id  object
dtypes: int64(2), object(6)
memory usage: 164.8+ MB


Esperamos tener los 3 DataFrames con 8 columnas, y aproximadamente 2,700,000 registros, debemos verificar que las columnas de este DataFrame coincidan con las de los de otros estados para hacer una unión correcta.

Cargamos los demás DataFrames.

In [6]:
df_flo = leer_carpeta(mainpath + 'Florida')
df_nev = leer_carpeta(mainpath + 'Nevada')

## 1.3 Verificación de columnas

Verificamos que tengan las mismas columnas.

In [7]:
cols = df_cal.columns # Tomamos uno de los dfs para usar sus columnas como guía.
for colev in (df_nev.columns, df_flo.columns):
    if colev.equals(cols): # Comparamos las columnas de cada df con las del ejemplo, si alguna no coincide, lo hacemos saber.
        continue
    else:
        print(f"Las columnas {colev} no coinciden con {cols}")

## 1.4 Unión vertical

Las columnas son las mismas, así que hacemos una unión vertical.

In [8]:
df = pd.concat([df_cal, df_flo, df_nev], axis=0)

Verificamos el resultado.

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7350000 entries, 0 to 1799999
Data columns (total 8 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   user_id  object
 1   name     object
 2   time     int64 
 3   rating   int64 
 4   text     object
 5   pics     object
 6   resp     object
 7   gmap_id  object
dtypes: int64(2), object(6)
memory usage: 504.7+ MB


Verificamos las filas.

In [10]:
total = df_cal.shape[0] + df_nev.shape[0] + df_flo.shape[0]
if df.shape[0] != total: # Verificamos que las filas del DataFrame resultante sean la suma de las filas de los 3 que unimos.
    print("Faltan filas en el DataFrame")

## 1.5 Exportamos los datos para su uso

In [11]:
df.to_csv('../../Data/Google/reviews_merged.csv', index=False)